## Task : Predict the churn score for a website based on the features provided in the dataset.

In [20]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [21]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# train = pd.read_csv(r'F:\Projects\Hackathons\HE\churn_risk_rate\dataset\train.csv')
# test = pd.read_csv(r'F:\Projects\Hackathons\HE\churn_risk_rate\dataset\test.csv')

train = pd.read_csv(r'/content/drive/MyDrive/Projects/churn_risk_rate/dataset/train.csv')
test = pd.read_csv(r'/content/drive/MyDrive/Projects/churn_risk_rate/dataset/test.csv')


# Dropping unimportant columns
output = pd.DataFrame(columns = ['customer_id','churn_risk_score'])
output['customer_id'] = test['customer_id']
cols_to_drop = ['customer_id', 'Name', 'security_no']
train = train.drop(cols_to_drop, axis = 1)
test = test.drop(cols_to_drop, axis = 1)


### Imputation for Missing Values

In [23]:
from sklearn.impute import SimpleImputer
imputer_gender = SimpleImputer(missing_values = 'Unknown', strategy='most_frequent')
train.loc[:,'gender'] = imputer_gender.fit_transform(train.loc[:,'gender'].to_numpy().reshape(-1,1))

test.loc[:,'gender'] = imputer_gender.transform(test.loc[:,'gender'].to_numpy().reshape(-1,1))

In [24]:
from sklearn.impute import SimpleImputer
imputer_medium_of_operation = SimpleImputer(missing_values = '?', strategy='most_frequent')
train.loc[:,'medium_of_operation'] = imputer_medium_of_operation.fit_transform(train.loc[:,'medium_of_operation'].to_numpy().reshape(-1,1))
test.loc[:,'medium_of_operation'] = imputer_medium_of_operation.transform(test.loc[:,'medium_of_operation'].to_numpy().reshape(-1,1))

In [25]:
train. avg_frequency_login_days = train.avg_frequency_login_days.replace({'Error':np.nan})
train.avg_frequency_login_days = pd.to_numeric(train.avg_frequency_login_days)

test.avg_frequency_login_days = test.avg_frequency_login_days.replace({'Error':np.nan})
test.avg_frequency_login_days = pd.to_numeric(test.avg_frequency_login_days)

In [26]:
from sklearn.impute import SimpleImputer
imputer_avg_f_login_days = SimpleImputer(missing_values = np.nan, strategy='mean')
train.loc[:,'avg_frequency_login_days'] = imputer_avg_f_login_days.fit_transform(train.loc[:,'avg_frequency_login_days'].to_numpy().reshape(-1,1))

test.loc[:,'avg_frequency_login_days'] = imputer_avg_f_login_days.transform(test.loc[:,'avg_frequency_login_days'].to_numpy().reshape(-1,1))

In [27]:
for i in range(train.shape[0]): 
    if train.loc[i, 'joined_through_referral'] == '?' : 
        if train.loc[i, 'referral_id'] == 'xxxxxxxx' : 
            train.loc[i, 'joined_through_referral'] = 'No'
        else : 
            train.loc[i, 'joined_through_referral'] = 'Yes'

for i in range(test.shape[0]):           
    if test.loc[i, 'joined_through_referral'] == '?' : 
        if test.loc[i, 'referral_id'] == 'xxxxxxxx' : 
            test.loc[i, 'joined_through_referral'] = 'No'
        else : 
            test.loc[i, 'joined_through_referral'] = 'Yes'

In [28]:
cols  = ['region_category', 'preferred_offer_types']

imputer_cat = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
train.loc[:,cols] = imputer_cat.fit_transform(train.loc[:,cols])
test.loc[:,cols] = imputer_cat.fit_transform(test.loc[:,cols])

In [29]:
imputer_points_in_wallet = SimpleImputer(missing_values = np.nan, strategy = 'mean')

train.loc[:,'points_in_wallet'] = imputer_points_in_wallet.fit_transform(train.loc[:,'points_in_wallet'].to_numpy().reshape(-1,1))
test.loc[:,'points_in_wallet'] = imputer_points_in_wallet.fit_transform(test.loc[:,'points_in_wallet'].to_numpy().reshape(-1,1))

In [30]:
train = train.drop(['referral_id', 'joining_date', 'last_visit_time'], axis = 1)
test = test.drop(['referral_id', 'joining_date', 'last_visit_time'], axis = 1)

In [31]:
train.avg_time_spent = abs(train.avg_time_spent)
test.avg_time_spent = abs(test.avg_time_spent)

train.avg_frequency_login_days = abs(train.avg_frequency_login_days)
test.avg_frequency_login_days = abs(test.avg_frequency_login_days)

for i in range(train.shape[0]) :
    if train.loc[i,'days_since_last_login'] < 0 : 
        train.loc[i,'days_since_last_login'] = 0

for i in range(test.shape[0]) : 
    if test.loc[i,'days_since_last_login'] < 0 : 
        test.loc[i,'days_since_last_login'] = 0

# Encoding

In [32]:
X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
X_test = test

In [33]:
cols_2_onehotencode = ['gender', 'region_category', 'membership_category',
       'joined_through_referral', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback']


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),cols_2_onehotencode)], remainder='passthrough')
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)


In [34]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [35]:
# y_train = y_train.replace(-1,0)

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from datetime import datetime

start = datetime.now()

param = { 'bootstrap': [True, False],
          'max_depth': [40,50,60,None],
          'min_samples_leaf': [1, 3, 5],
          'min_samples_split': [2, 4, 6],
          'n_estimators': [150, 200, 250]}

grids = GridSearchCV(RandomForestClassifier(max_features = 'sqrt'), param_grid = param, cv = 5, scoring='f1_weighted', n_jobs = 10, verbose = 10).fit(X_train, y_train)

end = datetime.now()

# print(cross_val_score(grids, X_train, y_train, cv = 5, scoring = 'f1_weighted'))

print('------| Time to fit the model : {} |------'.format(end - start))

print('------| Best paramters : {} |------'.format(grids.best_params_))

print('------| Best score : {} |------'.format(grids.best_score_))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   40.2s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  2.8min
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:  4.5min
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  5.7min
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:  7.5min
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:  9.1min
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed: 11.0min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed: 13.1min
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed: 15.2min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed: 17.7min
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed: 20.2min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed: 23.3min
[Parallel(n_jobs=10)]: Done 161 tasks      | elapsed: 26.6min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed: 29.3min
[Parallel(n_jobs=10)]: Done 201 tasks      | elapsed: 3

------| Time to fit the model : 3:36:22.102927 |------
------| Best paramters : {'bootstrap': False, 'max_depth': 50, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 150} |------
------| Best score : 0.7368295902917655 |------


In [37]:
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import cross_val_score

# cross_val_score(model,X_train, y_train,cv = 10, scoring='f1_weighted')

In [38]:
y_pred = grids.predict(X_test)

In [39]:
output['churn_risk_score'] = y_pred

In [40]:
# output.to_csv('F:\Projects\Hackathons\HE\churn_risk_rate\outputs\out1.csv', header = True, index = False)

output.to_csv('/content/drive/MyDrive/Projects/churn_risk_rate/outputs/out2.csv', header = True, index = False)

In [41]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, model.predict(X_train))

NameError: ignored